# Synthetic population approch

In [2]:
import pandas as pd
import numpy as np
from itertools import product

# Datei laden
df = pd.read_csv("../data/dazubi_grouped_berufe.csv")

# Relevante Variablen extrahieren
years = df['Jahr'].unique()
regions = df['Region'].unique()
berufe = df['Beruf_clean'].unique()
ages = ['16 und jünger', '17.0', '18.0', '19.0', '20.0', '21.0', '22.0', '23.0', '24 bis 39', '40 und älter']

# Altersverteilung extrahieren (Spaltennamen im Original)
age_cols = ['im Alter von: ' + a for a in ages]

# Basisstruktur für synthetische Population
synthetic_population = []

for _, row in df.iterrows():
    jahr = row['Jahr']
    region = row['Region']
    beruf = row['Beruf_clean']
    
    # Für jede Altersgruppe die Anzahl der Personen simulieren
    for age_label in age_cols:
        anzahl = int(row.get(age_label, 0))
        for _ in range(anzahl):
            synthetic_population.append({
                'Jahr': jahr,
                'Region': region,
                'Beruf': beruf,
                'Alter': age_label.replace('im Alter von: ', '')
            })

# In DataFrame umwandeln
synthetic_df = pd.DataFrame(synthetic_population)

# Optional speichern
# synthetic_df.to_csv("synthetic_population.csv", index=False)

# Beispiel: Vorschau
print(synthetic_df.sample(5))

         Jahr               Region                                      Beruf  \
736625   2011               Bremen             Beton- und Stahlbetonbauer/-in   
2263119  2014               Bayern                Tourismuskaufmann/-kauffrau   
2970435  2015  Nordrhein-Westfalen          Kaufmann/Kauffrau im Einzelhandel   
1283403  2012              Hamburg  Fachinformatiker/-in FR Systemintegration   
3203079  2016               Bayern                    Hochbaufacharbeiter/-in   

                 Alter  
736625            19.0  
2263119           17.0  
2970435      24 bis 39  
1283403      24 bis 39  
3203079  16 und jünger  


In [3]:
# CSV laden
df = pd.read_csv("../data/dazubi_grouped_berufe.csv")

# Altersgruppen vorbereiten
ages = ['16 und jünger', '17.0', '18.0', '19.0', '20.0', '21.0', '22.0', '23.0', '24 bis 39', '40 und älter']
age_cols = ['im Alter von: ' + a for a in ages]

# Leere Liste für synthetische Bevölkerung
synthetic_population = []

# Durch jede Zeile iterieren
for _, row in df.iterrows():
    jahr = row['Jahr']
    region = row['Region']
    beruf = row['Beruf_clean']
    
    # Anzahl Neuabschlüsse (alle Geschlechter)
    neuvertraege = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0)) + 100  # grober Default, falls leer
    geloeste_vertraege = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0))
    
    # Dropout-Risiko berechnen
    dropout_rate = geloeste_vertraege / neuvertraege if neuvertraege > 0 else 0

    # Altersverteilung berücksichtigen
    for age_label in age_cols:
        n = int(row.get(age_label, 0))
        for _ in range(n):
            geloest = np.random.rand() < dropout_rate  # zufällig "gedroppt" je nach Risiko
            synthetic_population.append({
                'Jahr': jahr,
                'Region': region,
                'Beruf': beruf,
                'Alter': age_label.replace('im Alter von: ', ''),
                'Vertragsart': 'neu abgeschlossen',
                'Vertragsstatus': 'beendet' if geloest else 'laufend',
                'Dropout_Risiko': round(dropout_rate, 3)
            })

# In DataFrame umwandeln
synthetic_df = pd.DataFrame(synthetic_population)

# Vorschau
print(synthetic_df.sample(5))

         Jahr             Region  \
2603657  2014          Thüringen   
5728347  2021            Hamburg   
2662402  2015  Baden-Württemberg   
5490827  2020    Rheinland-Pfalz   
4020875  2017    Rheinland-Pfalz   

                                                     Beruf          Alter  \
2603657                                         Bäcker/-in  16 und jünger   
5728347               Kaufmann/Kauffrau für Büromanagement      24 bis 39   
2662402                                  Mechatroniker/-in           17.0   
5490827  Technische/-r Produktdesigner/-in FR Maschinen...           18.0   
4020875                        Industriekaufmann/-kauffrau           18.0   

               Vertragsart Vertragsstatus  Dropout_Risiko  
2603657  neu abgeschlossen        laufend           0.130  
5728347  neu abgeschlossen        beendet           0.703  
2662402  neu abgeschlossen        laufend           0.457  
5490827  neu abgeschlossen        laufend           0.107  
4020875  neu abgesch

In [4]:
def simulate_age(age_label):
    if "16 und jünger" in age_label:
        return np.random.choice([14, 15, 16])
    elif "24 bis 39" in age_label:
        return np.random.randint(24, 40)
    elif "40 und älter" in age_label:
        return np.random.randint(40, 61)  # z. B. bis 60
    else:
        return int(float(age_label))  # z. B. "17.0" → 17

In [5]:
alter = simulate_age(age_label.replace("im Alter von: ", ""))

In [6]:
synthetic_df.to_csv("synthetische_population.csv", index=False)

In [7]:
# Datei laden
df = pd.read_csv("../data/dazubi_grouped_berufe.csv")

# Alterskategorien
ages = ['16 und jünger', '17.0', '18.0', '19.0', '20.0', '21.0', '22.0', '23.0', '24 bis 39', '40 und älter']
age_cols = ['im Alter von: ' + a for a in ages]

# Altersfunktion
def simulate_age(age_label):
    if "16 und jünger" in age_label:
        return np.random.choice([14, 15, 16])
    elif "24 bis 39" in age_label:
        return np.random.randint(24, 40)
    elif "40 und älter" in age_label:
        return np.random.randint(40, 61)
    else:
        return int(float(age_label))

# Leerer Container
synthetic_population = []

# Datengenerierung
for _, row in df.iterrows():
    jahr = row['Jahr']
    region = row['Region']
    beruf = row['Beruf_clean']

    # Dropout-Risiko
    neu = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0)) + 100
    abbruch = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0))
    dropout_rate = abbruch / neu if neu > 0 else 0

    # Geschlechterverteilung
    maenner_gesamt = sum([row.get(col, 0) for col in ['Deutsche Männer', 'Ausländer/-innen Männer']])
    frauen_gesamt = sum([row.get(col, 0) for col in ['Deutsche Frauen', 'Ausländer/-innen Frauen']])
    gesamt = maenner_gesamt + frauen_gesamt

    if gesamt > 0:
        p_mann = maenner_gesamt / gesamt
    else:
        p_mann = 0.5  # fallback

    # Iteration über Altersgruppen
    for age_label in age_cols:
        anzahl = int(row.get(age_label, 0))
        for _ in range(anzahl):
            alter = simulate_age(age_label.replace("im Alter von: ", ""))
            geschlecht = 'männlich' if np.random.rand() < p_mann else 'weiblich'
            status = 'beendet' if np.random.rand() < dropout_rate else 'laufend'

            synthetic_population.append({
                'Jahr': jahr,
                'Region': region,
                'Beruf': beruf,
                'Alter': alter,
                'Geschlecht': geschlecht,
                'Vertragsart': 'neu abgeschlossen',
                'Vertragsstatus': status,
                'Dropout_Risiko': round(dropout_rate, 3)
            })

# DataFrame erstellen
synthetic_df = pd.DataFrame(synthetic_population)

# Optional: als CSV speichern
synthetic_df.to_csv("synthetische_population.csv", index=False)

In [9]:
# CSV laden
df = pd.read_csv("../data/dazubi_grouped_berufe.csv")

# Altersgruppen
ages = ['16 und jünger', '17.0', '18.0', '19.0', '20.0', '21.0', '22.0', '23.0', '24 bis 39', '40 und älter']
age_cols = ['im Alter von: ' + a for a in ages]

# Funktion für Einzelalter
def simulate_age(age_label):
    if "16 und jünger" in age_label:
        return np.random.choice([14, 15, 16])
    elif "24 bis 39" in age_label:
        return np.random.randint(24, 40)
    elif "40 und älter" in age_label:
        return np.random.randint(40, 61)
    else:
        return int(float(age_label))

# Leere Liste
synthetic_population = []

# Schleife über Datensätze
for _, row in df.iterrows():
    jahr = row['Jahr']
    region = row['Region']
    beruf = row['Beruf_clean']

    # Dropout-Risiko
    neu = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0)) + 100
    abbruch = int(row.get('Vorzeitige Vertragslösungen Insgesamt', 0))
    dropout_rate = abbruch / neu if neu > 0 else 0

    # Geschlechterverteilung
    m_de = row.get('Deutsche Männer', 0)
    f_de = row.get('Deutsche Frauen', 0)
    m_ausl = row.get('Ausländer/-innen Männer', 0)
    f_ausl = row.get('Ausländer/-innen Frauen', 0)

    total = m_de + f_de + m_ausl + f_ausl

    for age_label in age_cols:
        anzahl = int(row.get(age_label, 0))

        for _ in range(anzahl):
            alter = simulate_age(age_label.replace("im Alter von: ", ""))

            # Geschlecht zufällig basierend auf Verteilung
            p_m = (m_de + m_ausl) / total if total > 0 else 0.5
            geschlecht = 'männlich' if np.random.rand() < p_m else 'weiblich'

            # Nationalität zufällig basierend auf Verteilung
            if geschlecht == 'männlich':
                p_de = m_de / (m_de + m_ausl) if (m_de + m_ausl) > 0 else 0.5
            else:
                p_de = f_de / (f_de + f_ausl) if (f_de + f_ausl) > 0 else 0.5

            nationalitaet = 'deutsch' if np.random.rand() < p_de else 'ausländisch'

            # Vertragsstatus
            status = 'beendet' if np.random.rand() < dropout_rate else 'laufend'

            synthetic_population.append({
                'Jahr': jahr,
                'Region': region,
                'Beruf': beruf,
                'Alter': alter,
                'Geschlecht': geschlecht,
                'Nationalität': nationalitaet,
                'Vertragsart': 'neu abgeschlossen',
                'Vertragsstatus': status,
                'Dropout_Risiko': round(dropout_rate, 3)
            })

# DataFrame erstellen
synthetic_df = pd.DataFrame(synthetic_population)

# Speichern
synthetic_df.to_csv("synthetische_population.csv", index=False)

In [ ]:
t